# House Price Prediction2
I decided to Fork the Notebook and  change some of the features.(log of saleprice).
Scaling the Saleprice seems to have a very positive impact on the acc. of the models.

# Imports

In [ ]:
import numpy as np 
import pandas as pd 


# plot
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import  norm
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import RobustScaler 
#This Scaler removes the median and scales the data according to the quantile range
# better for models like SVR

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# import warnings 
# warnings.filterwarnings('ignore')

print("imports sucessful")

# Data Processing

In [ ]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv", index_col="Id")
test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv", index_col="Id")
train

In [ ]:
print("Train.shape: ", train.shape, "\tTest.shape: ", test.shape)

In [ ]:
trainY=train.pop("SalePrice")
trainY

In [ ]:
trainLen=len(train)
allData=pd.concat((train,test)).reset_index(drop=True)
allData

We first want to find and get rid of NaN

In [ ]:
objCols = allData.select_dtypes(include=['object']).columns
numCols =allData.select_dtypes(exclude=['object']).columns
objCols

In [ ]:
numCols

In [ ]:
print("len num: ", len(numCols),"\tlen num: ", len(objCols))  #a+b=79   so check if we have all in a category

In [ ]:
nan=[col for col in allData.columns if allData[col].isnull().sum()>0] #List of Cols with nan
print(nan)

In [ ]:
print(allData['MasVnrType'].isnull().sum(),allData["MasVnrArea"].isnull().sum()) # =no type=> no area? =>no mas..

In [ ]:
print(allData['GarageType'].isnull().sum(),allData["GarageYrBlt"].isnull().sum())  #similar => no garage=> no jear built

In [ ]:
allData['KitchenQual'].isnull().sum()  # 1 missing   maybe no kitchen

In [ ]:
# a nan in fireplace. pool etc => no pool/fireplace etc.  Exterior2nd=Nan=probl just 1 material    no basement parameters=> probl no basement...
# you can get information from the data_description.txt
# these assumtions might be false sometime, but the results should be much better than dropping important cols like pool
cat=["Fence","PoolQC", "MiscFeature", "Alley",'FireplaceQu', 'MasVnrType',"Exterior2nd", "BsmtCond","BsmtExposure","BsmtFinType1", "BsmtFinType2", "KitchenQual", 
   "GarageType", "GarageFinish", "GarageQual", 'GarageCond', "BsmtQual"]#fill with None
num=["BsmtFinSF1","MasVnrArea","BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageYrBlt", "GarageCars", "GarageArea", ] #fill with 0
for i in cat:
    allData[i].fillna(value="None", inplace=True)
for i in num:
    allData[i].fillna(value=0, inplace=True)

In [ ]:
nan=[col for col in allData.columns if allData[col].isnull().sum()>0] #List of Cols with nan   after filling in stuff
print(nan) # these are the Parameters that you cant fill as easily(as far as i can see)

In [ ]:
for i in nan:
    print(allData[i].isnull().sum())  # only LotFrontage has a lot of missing vals

In [ ]:
tiny=['MSZoning', 'Utilities', 'Exterior1st', 'Electrical', 'Functional','SaleType']
allData[tiny] = allData[tiny].fillna(allData.mode().iloc[0])  #fill cat with most common

In [ ]:
nan=[col for col in allData.columns if allData[col].isnull().sum()>0] #List of Cols with nan   after filling in stuff
nan # LotFrontage is numerical  = Linear feet of street connected to property
#there are a lot of missing vals   I don't think this parametier is very Important to the price of a House but im not an expert

In [ ]:
allData["LotFrontage"] = allData["LotFrontage"].fillna(allData["LotFrontage"].mean())  #fill with mean

now that we have gotten rid of NaN we have to further prep the data

In [ ]:
numCols

notes: <br>
- Maybe we could change YrSold and similar parameters to sold x years ago, which could improve the performance, but I dont't know what time the data has been collected. I will use 2020 as the "now" time. 
- I decide not to combine month and year. Month on its own shows which season the sale took place which could have an impact.

In [ ]:
# new cols, that might get insight/make it more obvious
allData["SoldYrAgo"]=2020-allData["YrSold"]
allData["GarageYrBltAgo"]=2020-allData["GarageYrBlt"]
allData["BuiltAgo"]=2020-allData["YearBuilt"]
allData["SoldYrAgo"]

In [ ]:
#creating the data to plot with later   before scaling... happens
plotdata=allData[:trainLen]
plotdata["Saleprice"]=trainY

In [ ]:
#onehot cols that are not linear
# i dont replace all categorical cols, because some should work fine in 1 col, i m looking at the data_description
neededOneHot=["MSZoning", "MSSubClass", "LandContour", "Utilities", "LotConfig", "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", 
       "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType", "ExterCond",  "Foundation",  "Heating", "Electrical", "GarageType", "MiscFeature",
        "SaleType", "SaleCondition"]

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
for col in neededOneHot:
    allData=pd.concat([allData,pd.get_dummies(allData[col],prefix=col)], axis=1)
    allData.drop([col],axis=1, inplace=True)

In [ ]:
allData.shape

In [ ]:
allData.BsmtFinType2.unique()

In [ ]:
# manually concerting "quality" parameters
trans = {"Grvl": 0, "Pave": 1, "None": 2,}
allData['Alley']= allData['Alley'].map(trans)

trans = {"Reg": 0, "IR1": 1, "IR2": 2, "IR3": 3, "None": 4}
allData['LotShape']= allData['LotShape'].map(trans)

trans = {"Gtl": 0, "Mod": 1, "Sev": 2, "None": 3}
allData['LandSlope']= allData['LandSlope'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['ExterQual']= allData['ExterQual'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4,  "None": 5}
allData['BsmtQual']= allData['BsmtQual'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4,  "None": 5}
allData['BsmtCond']= allData['BsmtCond'].map(trans)

trans = { "Gd": 1, "Av": 2, "Mn": 3, "No": 4,  "None": 5}
allData['BsmtExposure']= allData['BsmtExposure'].map(trans)

trans = {"GLQ": 0, "ALQ": 1, "BLQ": 2, "Rec": 3, "LwQ": 4,  "Unf": 5, "None": 6}
allData['BsmtFinType1']= allData['BsmtFinType1'].map(trans)

trans = {"GLQ": 0, "ALQ": 1, "BLQ": 2, "Rec": 3, "LwQ": 4,  "Unf": 5, "None": 6}
allData['BsmtFinType2']= allData['BsmtFinType2'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['HeatingQC']= allData['HeatingQC'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['KitchenQual']= allData['KitchenQual'].map(trans)

trans = {"Typ": 0, "Min1": 1, "Min2": 2, "Mod": 3, "Maj1": 4,  "Maj2": 5,"Sev": 6,"Sal": 7, "None": 8}
allData['Functional']= allData['Functional'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['FireplaceQu']= allData['FireplaceQu'].map(trans)

trans = {"Fin": 0, "RFn": 1, "Unf": 2,  "None": 3}
allData['GarageFinish']= allData['GarageFinish'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['GarageQual']= allData['GarageQual'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['GarageCond']= allData['GarageCond'].map(trans)

trans = {"Ex": 0, "Gd": 1, "TA": 2, "Fa": 3, "Po": 4, "None": 5}
allData['PoolQC']= allData['PoolQC'].map(trans)

trans = {"GdPrv": 0, "MnPrv": 1, "GdWo": 2, "MnWw": 3,  "None": 5}
allData['Fence']= allData['Fence'].map(trans)

In [ ]:
#redo after onehot etc
objCols = allData.select_dtypes(include=['object']).columns
numCols =allData.select_dtypes(exclude=['object']).columns
l=[]
for i in objCols: #showing cat data example
    l.append(allData[i][0:5])
print(l)

In [ ]:
#changing cat. data to num
for i in objCols:
    allData[i]= allData[i].astype('category').cat.codes

l=[]
for i in objCols: #showing they changed to num
    l.append(allData[i][0:5])
print(l)   

In [ ]:
#drop the cols we have twice
allData.drop("YrSold", axis=1, inplace=True)
allData.drop("GarageYrBltAgo", axis=1, inplace=True)
allData.drop("BuiltAgo", axis=1, inplace=True)

In [ ]:
allData.shape #the shape stays the same

In [ ]:
scaler = MinMaxScaler()
for col in allData.columns:
    allData[col] = scaler.fit_transform(allData[col].values.reshape(-1,1))

In [ ]:
allData

notes:
I don't think I should remove any col.

In [ ]:
trainY

In [ ]:
# log of saleprice as new saleprice
trainY=np.log1p(trainY)

In [ ]:
trainY

# Plot
I don't know what I want to plot as the nr. of cols is overwhelming and I don't know much about House-Prices. I will edit the data if I see it is needed in here.

In [ ]:
# #attention this will take a while
# plt.figure(figsize=(64,48))
# sns.heatmap(plotdata.corr(),annot=True,cmap='Reds')
# plt.show()# a lot of small correlations

As seen on the Heatmap, the correlations between the parameters and Saleprice are very small.<br>
It is also notable that YearBuilt and BuiltAgo for example have the same correlation, which is logical as they say the same thing

In [ ]:
plotdata.columns#seel what I can plot with

Looking at the distribution of some cols

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(plotdata['Saleprice'],bins=30,color='m', fit=norm)
fig = plt.figure()
res = stats.probplot(plotdata['Saleprice'], plot=plt)
# prob. plot from https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [ ]:
# a lot of ml models work better with norm dist.
plotdata["SalepriceLog"] = np.log1p(plotdata["Saleprice"])
# allData["Saleprice"] = np.log1p(allData["Saleprice"])

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(plotdata['SalepriceLog'],bins=30,color='m', fit=norm)
fig = plt.figure()
res = stats.probplot(plotdata['SalepriceLog'], plot=plt)# much better
# need to np.expm1() the output to undo the log. for submitting results

In [ ]:
plt.figure(figsize=(8,5))# not a used col
sns.distplot(plotdata['BuiltAgo'],bins=30, fit=norm)# most houses are not that old

In [ ]:
# a lot of ml models work better with norm dist.
plotdata["BuiltAgo"] = np.log1p(plotdata["BuiltAgo"])
plt.figure(figsize=(8,5))
sns.distplot(plotdata['BuiltAgo'],bins=30, fit=norm)# dont know if this is better as it has several bumps I dont change the allData

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(plotdata['YearBuilt'],bins=30, fit=norm,color='r')# most houses are not that old

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(plotdata['GarageArea'],bins=30, fit=norm,color='g')

In [ ]:
plotdata["GarageArea"] = np.log1p(plotdata["GarageArea"])
plt.figure(figsize=(8,5))
sns.distplot(plotdata['GarageArea'],bins=30, fit=norm,color='g')# much worse?

In [ ]:
plt.figure(figsize=(8,5))
sns.distplot(plotdata['GrLivArea'],bins=30, fit=norm,color='m')# much worse

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="GrLivArea",y="Saleprice",  data=plotdata)

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="BuiltAgo",y="Saleprice",hue="Fireplaces",  data=plotdata)# fireplaces are nice, but they are not that hard to build if the house does not have one already

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="MSZoning",y="Saleprice",  data=plotdata)# some are clearly "cheaper"

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="GrLivArea",y="Saleprice",  data=plotdata,color='m') 

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="GarageArea",y="Saleprice",hue="GarageCond",  data=plotdata)

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="GarageArea",y="Saleprice",hue="GarageQual",  data=plotdata)

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4,figsize=(15,5))
plt.figure(figsize=(80,50))
fig.suptitle('Garage')
ax1.scatter(x="GarageArea",y="Saleprice",  data=plotdata, linewidth=1)
ax2.scatter(x="GarageQual",y="Saleprice",  data=plotdata, linewidth=1)
ax3.scatter(x="GarageCond",y="Saleprice",  data=plotdata, linewidth=1)
ax4.scatter(x="GarageFinish",y="Saleprice",  data=plotdata, linewidth=1)

fig.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="PoolArea",y="Saleprice",hue="PoolQC",  data=plotdata)# a pool does not seem to make a house very expensive interestingly

In [ ]:
plt.figure(figsize=(8,5))
sns.scatterplot(x="SoldYrAgo",y="Saleprice", data=plotdata,color='g')#older sales=> more expensive  (bubble?)

# Models
testing a lot of several models.

In [ ]:
#final nan check
nan=[col for col in allData.columns if allData[col].isnull().sum()>0] #List of Cols with nan   after filling in stuff
print(nan)

In [ ]:
#data for the models
trainX=np.array(allData[:trainLen])
testX=np.array(allData[trainLen:])
trainY=np.array(trainY)

# Linear regression

there is a strange bug, wehre saleprice with index 1444 will return inf. But if I use the Robustscaler it works.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# modelLin = LinearRegression()
# modelLin.fit(trainX, trainY)
# predTrain=np.expm1(modelLin.predict(trainX))
# LinAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(LinAcc) + ' MSE') #Score= failed(bug?)

In [ ]:
modelLin = make_pipeline(RobustScaler(), LinearRegression())
modelLin.fit(trainX, trainY)
predTrain=np.expm1(modelLin.predict(trainX))
LinAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(LinAcc) + ' MSE') #Score= 0.14579

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

the score seems to be very bad

# SVR

In [ ]:
from sklearn.svm import SVR

manually tunning

In [ ]:
# kernels: poly, rbf
modelSvr = make_pipeline(RobustScaler(), SVR(kernel="poly", C=1, degree=1))    # is (almost) like linReg when 1degree...
modelSvr.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelSvr.predict(trainX))
SVRAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(SVRAcc) + ' MSE') # huge mse   score=0.13235   #very diffrent than houseprice1

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

# XGBoost

In [ ]:
import xgboost

Hypertunning 1<br>
attention, the hypertunning may take a very long time(don't run this for your own sake)

In [ ]:
# #xgboost
# modelX=xgboost.XGBRegressor(verbose=2)

# n_estimators = [500, 900, 2000] # a lot of cols=> many estim..
# max_depth = [3, 6,9,12]
# booster=['gbtree','gblinear'] #"dart"
# learning_rate=[.02,.05,.1,.2]
# min_child_weight=[2,5,9]
# base_score=[.01,.25,.5,.75]
# max_depth=[2,4,6,8]
# min_split_loss=[0,0.01,.1]
# hyperparameter_grid = {
#     'n_estimators': n_estimators,
#     'max_depth':max_depth,
#     'learning_rate':learning_rate,
#     'min_child_weight':min_child_weight,
#     'booster':booster,
#     'base_score':base_score,
#     "max_depth":max_depth,
#     "min_split_loss":min_split_loss}

# random_cv = RandomizedSearchCV(estimator=modelX,
#             param_distributions=hyperparameter_grid,
#             cv=5, n_iter=100,
#             scoring='neg_root_mean_squared_error',
#                                n_jobs = -1,
#             verbose = 5, 
#             return_train_score = True,
#             random_state=42)

# random_cv.fit(trainX,trainY)

In [ ]:
# random_cv.best_estimator_    #if best estim are at the edge=> additional tests

In [ ]:
# modelX=xgboost.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
#              importance_type='gain', interaction_constraints='',
#              learning_rate=0.02, max_delta_step=0, max_depth=8,
#              min_child_weight=9, min_split_loss=0, missing=None,
#              monotone_constraints='()', n_estimators=2000, n_jobs=0,
#              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
#              scale_pos_weight=1, subsample=1, tree_method='exact',
#              validate_parameters=1,  verbosity=None)#Score= 0.12949

In [ ]:
# modelX=xgboost.XGBRegressor(base_score=0.01, booster='gbtree', colsample_bylevel=1,
#              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
#              importance_type='gain', interaction_constraints='',
#              learning_rate=0.05, max_delta_step=0, max_depth=4,
#              min_child_weight=5, min_split_loss=0, missing=None,
#              monotone_constraints='()', n_estimators=500, n_jobs=0,
#              num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
#              scale_pos_weight=1, subsample=1, tree_method='exact',
#              validate_parameters=1, verbose=2, verbosity=None)#  score = 0.12808

In [ ]:
# modelX.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelX.predict(trainX))
# XAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(XAcc) + ' MSE') #Score= 0.12808

manually tunning<br>
after having seen the score of the light GBM i wanted to atleaset go below a score of 0.13

In [ ]:
modelX=xgboost.XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.015, max_delta_step=0, max_depth=4,
             min_child_weight=5, min_split_loss=0, missing=None,
             monotone_constraints='()', n_estimators=3000, n_jobs=0,
             num_parallel_tree=1, random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1,  verbosity=None)
modelX.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelX.predict(trainX))
XAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(XAcc) + ' MSE') #Score= 0.12670

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

# Random Forrest

In [ ]:
from sklearn.ensemble import  RandomForestRegressor

Hypertunning 1<br>
attention, the hypertunning may take a long time

In [ ]:
# modelForrest=RandomForestRegressor()# not regressor

# #Hypertunning    from https://www.kaggle.com/harshkothari21/beginners-notebook-90-accuracy
# n_estimators = [100,300, 500, 1000]
# depth = [3,5,10,15]
# min_split=[2,3,4]
# min_leaf=[2,3,4]
# bootstrap = ['True', 'False']
# verbose = [5]

# hyperparameter = {
#     'n_estimators': n_estimators,
#     'max_depth':depth,
#     'bootstrap':bootstrap,
#     'verbose':verbose,
#     'min_samples_split':min_split,
#     'min_samples_leaf':min_leaf    }

# modelForrest = RandomizedSearchCV(estimator=modelForrest,
#                                param_distributions=hyperparameter,
#                                cv=5, #n_iter=100,
#                                scoring = 'neg_mean_absolute_error',
#                                n_jobs = 4, 
#                                return_train_score = True,
#                                random_state=1)
# modelForrest.fit(trainX,trainY)

In [ ]:
# modelForrest.best_estimator_    #if best estim are at the edge=> additional tests

In [ ]:
# modelForrest=RandomForestRegressor(bootstrap='False', max_depth=15, min_samples_leaf=3,
#                       verbose=1)
# modelForrest.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelForrest.predict(trainX))
# ForrestAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(ForrestAcc) + ' MSE') #Score= 0.14578

Hypertunning 2<br>
manually tuning

In [ ]:
modelForrest=RandomForestRegressor(n_estimators=1100, bootstrap='True', max_depth=14, min_samples_leaf=3,
                      verbose=1)
modelForrest.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelForrest.predict(trainX))
ForrestAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(ForrestAcc) + ' MSE') #Score= 0.14492

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

# Light Gradient Boosting

In [ ]:
from lightgbm import LGBMRegressor

Hypertunning 1 attention, the hypertunning may take along time <br>
(I don't know much about light gbm -> can't hypertune very well)

In [ ]:
# modelLight=LGBMRegressor()

# n_estimators = [ 700, 1000, 1500, 2000, 3000]
# learning_rate=[.02,.05,.1,.15,.20]
# max_depth =[0,3, 5, 8]
# num_leaves =[16,25,31]  #Maximum tree leaves

# hyperparameter_grid = {
#     'n_estimators': n_estimators,
#     'learning_rate':learning_rate,
#     'num_leaves':num_leaves,
#     "max_depth": max_depth}

# modelLight = RandomizedSearchCV(estimator=modelLight,
#             param_distributions=hyperparameter_grid,
#             cv=5, n_iter=50,
#             scoring = 'neg_mean_absolute_error',n_jobs = 4,
#             verbose = 5, 
#             return_train_score = True,
#             random_state=42)
# modelLight.fit(trainX,trainY)

In [ ]:
# modelLight.best_estimator_    #if best estim are at the edge=> additional tests

In [ ]:
# modelLight =LGBMRegressor(learning_rate=0.05, max_depth=3, n_estimators=700, num_leaves=25)
# modelLight.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelLight.predict(trainX))
# LightAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(LightAcc) + ' MSE') #Score= don't trust this one

Manual<br>

In [ ]:
# modelLight =LGBMRegressor(learning_rate=0.02, max_depth=4, n_estimators=700*10, num_leaves=12)
# modelLight.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelLight.predict(trainX))
# LightAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(LightAcc) + ' MSE') #Score= 0.12763

As I don't know much about light gbm I also looked at another notebook  https://www.kaggle.com/jesucristo/1-house-prices-solution-top-1

In [ ]:
modelLight = LGBMRegressor(objective='regression', 
                       num_leaves=6,
                       learning_rate=0.01, 
                       n_estimators=7000,
                       max_bin=200, 
                       bagging_fraction=0.8,
                       bagging_freq=4, 
                       bagging_seed=8,
                       feature_fraction=0.2,
                       feature_fraction_seed=8,
                       min_sum_hessian_in_leaf = 11,
                       verbose=-1,
                       random_state=42)
modelLight.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelLight.predict(trainX))
LightAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(LightAcc) + ' MSE') #Score= 0.12185

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

# Gradient Boost

In [ ]:
from sklearn.ensemble import  GradientBoostingRegressor

Hypertunning 1
attention, the hypertunning may take a very long time

In [ ]:
# modelGrad=GradientBoostingRegressor()

# n_estimators = [ 700, 1000, 1500, 2000, 3000]
# loss=[ "huber"]
# criterion=["friedman_mse"]
# learning_rate=[.02,.05,.1,.15,.20]
# min_samples_split=[1,3, 5, 8]
# min_samples_leaf=[1, 2, 3, 4, 5]
# hyperparameter_grid = {
#     'n_estimators': n_estimators,
#     'learning_rate':learning_rate,
#     'min_samples_split':min_samples_split,
#     "loss" :loss,
#     'min_samples_leaf':min_samples_leaf,
#     "criterion": criterion}

# modelGrad = RandomizedSearchCV(estimator=modelGrad,
#             param_distributions=hyperparameter_grid,
#             cv=5, n_iter=50,
#             scoring = 'neg_mean_absolute_error',n_jobs = 4,
#             verbose = 5, 
#             return_train_score = True,
#             random_state=42)
# modelGrad.fit(trainX,trainY)

In [ ]:
# modelGrad.best_estimator_    #if best estim are at the edge=> additional tests

In [ ]:
modelGrad=GradientBoostingRegressor(learning_rate=0.02, loss='huber', min_samples_leaf=2,
                          min_samples_split=5, n_estimators=2000)
modelGrad.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelGrad.predict(trainX))
GradAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(GradAcc) + ' MSE') #Score= 0.12964

manual tunning

In [ ]:
# modelGrad=GradientBoostingRegressor(learning_rate=0.01, loss='huber', min_samples_split=3,
#                           n_estimators=4000)
# modelGrad.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelGrad.predict(trainX))
# GradAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(GradAcc) + ' MSE') #Score= 0.13006   did not beat randomized...

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

# Adaboost

In [ ]:
from sklearn.ensemble import  AdaBoostRegressor

Hypertunning 1 <br>

In [ ]:
# modelAda=AdaBoostRegressor()

# n_estimators = [ 700, 1000, 1500, 2000, 3000]
# loss=[ "linear", "square"]
# learning_rate=[.02,.1,.15,.60, 1]

# hyperparameter_grid = {
#     'n_estimators': n_estimators,
#     'learning_rate':learning_rate,
#     "loss" :loss,
#     }

# modelAda = RandomizedSearchCV(estimator=modelAda,
#             param_distributions=hyperparameter_grid,
#             cv=5, n_iter=50,
#             scoring = 'neg_mean_absolute_error',n_jobs = 4,
#             verbose = 5, 
#             return_train_score = True,
#             random_state=42)
# modelAda.fit(trainX,trainY)

In [ ]:
# modelAda.best_estimator_    #if best estim are at the edge=> additional tests

In [ ]:
modelAda=AdaBoostRegressor(learning_rate=0.02, n_estimators=1000)
modelAda.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelAda.predict(trainX))
AdaAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(AdaAcc) + ' MSE') #Score= 0.17866

manual tunning

In [ ]:
# modelAda=AdaBoostRegressor(learning_rate=0.01, n_estimators=2500)
# modelAda.fit(trainX,trainY)

In [ ]:
# predTrain=np.expm1(modelAda.predict(trainX))
# AdaAcc = mean_squared_error(predTrain, np.expm1(trainY))
# print (str(AdaAcc) + ' MSE') #Score= 0.17873  did not preform better than randomized....

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

AdaBoost, RandomForrest and LinReg performed poorly

# Stacking the models

In [ ]:
from mlxtend.regressor import StackingCVRegressor

In [ ]:
# im testing with dif models
regressors = (modelLight
              #,modelLin
              #,modelForrest
              ,modelX
              ,modelSvr 
              #,modelAda 
              #,modelGrad
)
modelStack = StackingCVRegressor(regressors=regressors,
                                meta_regressor=modelLight,  # i trust xgboost/light gbm the most as they performed better than the other models
                                use_features_in_secondary=True)

In [ ]:
modelStack.fit(trainX,trainY)

In [ ]:
predTrain=np.expm1(modelStack.predict(trainX))
StAcc = mean_squared_error(predTrain, np.expm1(trainY))
print (str(StAcc) + ' MSE') #Score= 

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot((np.expm1(trainY)),bins=30, color='m')
sns.distplot((predTrain),bins=30, color='g')

notes:
- 1. try models used: modelForrest, modelX, modelLight, modelAda, modelSvr, modelLin, modelGrad.  Metamodel=modelX, score= 0.12755
- 2. try models used: modelX, modelLight, modelSvr, modelGrad.  Metamodel=modelX, score=0.12667
- 3. try models used: modelX, modelLight, modelSvr.  Metamodel=modelX, score=0.12291
- 4. try models used: modelX, modelLight.  Metamodel=modelX, score=0.13219
- 5. try models used: modelX, modelLight, modelSvr.  Metamodel=modelLight, score=0.12185


# averaging
taking the average of good performing models to see if results get better.

In [ ]:
# predictions=pd.DataFrame({'Id': test.index,
#                        'SalePrice': (np.expm1(modelX.predict(testX))+np.expm1(modelLight.predict(testX)))/2}) # score = 0.12201

# save Prediction

In [ ]:
predictions=pd.DataFrame({'Id': test.index,
                       'SalePrice': np.expm1(modelStack.predict(testX))})

In [ ]:
predictions

In [ ]:
# plot results
plt.figure(figsize=(12,5))
sns.distplot(predictions["SalePrice"],bins=30, color='m')

In [ ]:
predictions.to_csv("submit.csv", index=False)